In [1]:
!nvidia-smi

Tue Aug 16 02:08:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

CNN on full data

ADASYN+Tomek Link+XGBoost to balance data

In [2]:
from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
drive.mount('/content/gdrive')
!ls
%cd gdrive/My\ Drive/phd-thesis/
!ls

Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/phd-thesis
 adasyn_TL_MajorVoting_balance.ipynb
 ADASYN_TL_MajorVoting_imb.ipynb
 adasyn_xgboost_20_imb.ipynb
 adasyn_xgboost_20.ipynb
 adasyn_xgboost_fulldata_balance.ipynb
 adasyn_xgboost_fulldata_imb.ipynb
 Attack_Types.csv
'bagging&ensemble.py'
 cnn.ipynb
 CNN-test-data
 CNN-test-data-balanced
 CNN-test-data-balanced-dropout
 CNN-test-data-imb
 createCNN_Balanced_fulldata.ipynb
 createCNN_Balanced.ipynb
 createNetwork-211025.py
 createNetwork-CNN-imb.ipynb
'createNetwork-testdata (1).ipynb'
 createNetwork-testdata.ipynb
 createNetwork-testdata.py
 ensemble_balance-old.ipynb
 ensemble_imb-old.ipynb
 Evaluate_adasyn_xgboost_model_fulldata.ipynb
 Evaluate_adasyn_xgboost_model_fulldata.ipynb（副本）
 Evaluate_adasyn_xgboost_model.ipynb
 evaluate_auc_3Algorithm_macro.ipynb
 evaluate_auc_3Algorithm_micro.ipynb
 Evaluate_CNN_Balacanced.ipynb
 Evaluate_CNN_imb.ipynb
 Evaluate_PR_3Algorithms.ipynb
 Evaluate_PR_CNN.ipynb
 Eva

In [3]:
pip install --upgrade xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 83 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  8 10:12:36 2022
using ADASYN and tomek link, and xgboost
with 20% dataset
balanced data model on test data
@author: Administrator
"""

from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, zero_one_loss, classification_report, \
    plot_confusion_matrix
    

from collections import Counter
from sklearn.metrics import accuracy_score
import sys
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import xgboost as xgb
import multiprocessing
import joblib


#traing dataset
# train_x = pd.read_csv('KDDTrain_x_20.csv',header=None, nrows=1000)
# train_Y = pd.read_csv('KDDTrain_y_20.csv',header=None, nrows=1000)
train_x = pd.read_csv('KDDTrain_x.csv',header=None)
train_Y = pd.read_csv('KDDTrain_y.csv',header=None)
train_x.drop(train_x.columns[[115,116,117]], axis=1, inplace=True)
train_Y[0]=list(train_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))

print("train_x", train_x.head())
print("train_y", train_Y.head())

#use few dataset to test 
# train_x = train_x.head()
# train_Y = train_x.head()

# train and test data split
# x_train, x_test, y_train, y_test = train_test_split( \
     # train_x, train_Y, test_size=0.3, random_state=42)


#testing dataset
test_x = pd.read_csv('KDDTest_x.csv',header=None)
test_Y = pd.read_csv('KDDTest_y.csv',header=None)
test_x.drop(test_x.columns[[115,116,117]], axis=1, inplace=True)
test_Y[0]=list(test_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))
print("test_x", test_x.head())
print("test_y", test_Y.head())

# train_Y = train_Y.values.ravel()
# test_Y  = test_Y.values.ravel()
# print('Original dataset shape %s' % Counter(train_Y))

train_Y = train_Y.values.ravel()
test_Y  = test_Y.values.ravel()
print('Original train dataset shape %s' % Counter(train_Y))
print('Original test dataset shape %s' % Counter(test_Y))

######--------------- RandomUnderSampler-------------------------
mean_class_size = int(pd.Series(train_Y).value_counts().sum()/5)
print("mean_class_size", mean_class_size)

ratio= {0: mean_class_size,
        1: mean_class_size
        #'probe': mean_class_size,
        #'r2l': mean_class_size,
        #'u2r': mean_class_size
        }

# check if ratio para is still used
# print("before \n", pd.Series(train_Y).value_counts())
# print('Original dataset shape %s' % Counter(train_Y))

#from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=ratio, random_state=42)
train_x, train_Y = rus.fit_resample(train_x, train_Y)
print('now dataset shape %s' % Counter(train_Y))


# #only smote
# from imblearn.over_sampling import SMOTE
# sm = SMOTE (sampling_strategy='auto', random_state=0)
# # train_x_sm, train_Y_sm = sm.fit_resample(train_x, train_Y)
# train_x_sm, train_Y_sm = sm.fit_resample(train_x, train_Y)
# print('smote dataset shape %s' % Counter(train_Y_sm))

#*******try using ADASYN on test dataset to make balance
#only 
from imblearn.over_sampling import ADASYN
ad = ADASYN (sampling_strategy='not majority', random_state=0)
# train_x_sm, train_Y_sm = sm.fit_resample(train_x, train_Y)
train_x_ad, train_Y_ad = ad.fit_resample(train_x, train_Y)
print('adasyn dataset shape %s' % Counter(train_Y_ad))

#Tomek Links
from imblearn.under_sampling import TomekLinks
sampling_strategy = "all"
tl = TomekLinks(sampling_strategy=sampling_strategy)
# x_res, Y_res = tl.fit_resample(train_x_sm, train_Y_sm)
x_res, Y_res = tl.fit_resample(train_x_ad, train_Y_ad)
print('Tomek Links dataset shape %s' % Counter(Y_res))

#sys.exit(0)

### ENN
# from imblearn.under_sampling import EditedNearestNeighbours
# sampling_strategy = "not minority"
# enn = EditedNearestNeighbours(sampling_strategy=sampling_strategy)
# x_res, y_res = enn.fit_resample(train_x_sm, train_Y_sm)
# print('ENN dataset shape %s' % Counter(y_res))



"""######--------------- RandomUnderSampler-------------------------
mean_class_size = int(pd.Series(train_Y).value_counts().sum()/5)
print("mean_class_size", mean_class_size)

ratio= {'benign': mean_class_size,
        'dos': mean_class_size,
        'probe': mean_class_size,
        'r2l': mean_class_size,
        'u2r': mean_class_size
        }

# check if ratio para is still used
# print("before \n", pd.Series(train_Y).value_counts())
print('Original dataset shape %s' % Counter(train_Y))

rus = RandomUnderSampler(sampling_strategy=ratio, random_state=0)
train_x_rus, train_Y_rus = rus.fit_resample(train_x, train_Y)

print('now dataset shape %s' % Counter(train_Y_rus))
"""
"""
# decision tree clf
classifier = DecisionTreeClassifier(random_state=17)
classifier.fit(x_res, Y_res)

# pred_y = classifier.predict(test_x)
# results = confusion_matrix(test_Y, pred_y)
# error = zero_one_loss(test_Y, pred_y)

pred_y = classifier.predict(x_test)
results = confusion_matrix(y_test, pred_y)
error = zero_one_loss(y_test, pred_y)
"""

#***xgboost clf
#xgb_model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count() // 2)
#xgb_model = xgb.XGBClassifier(n_jobs=1).fit(x_res, Y_res)
#kf = KFold(n_splits=2, shuffle=True, random_state=rng)
# xgb_model = xgb.XGBClassifier(objective='multi:softmax')

xgb_model = xgb.XGBClassifier()
# clf = GridSearchCV(xgb_model, {'max_depth': [2, 4, 6],
#                                y'n_estimators': [50, 100, 200]}, verbose=1,
#                                n_jobs=2)
# param_grid = [
#     {'max_depth': [2, 4, 6], 'n_estimators': [50, 100, 200]}, 
#     {'objective': ['multi:softmax'], 'verbose': [1], 'n_jobs':[-1]},
#     ]
    
param_grid = [
    {'max_depth': [6], 'n_estimators': [200]}, 
    {'objective': ['multi:softprob'], 'num_class': [5], 'verbose': [1], 'n_jobs':[-1]},
    ]
    

# clf = GridSearchCV(xgb_model, {'max_depth': [2, 4, 6],
#                                'n_estimators': [50, 100, 200]}, verbose=1,
#                                 n_jobs=-1)
# clf = GridSearchCV(xgb_model, param_grid, cv=2)
clf = GridSearchCV(xgb_model, param_grid, cv=5)

clf.fit(x_res, Y_res)

# save xgboost model
clf.best_estimator_.save_model('myXgb_adasyn.json')

print(clf.best_score_)
print(clf.best_params_)




#evaluation with no sampling on test data
#pred_y = xgb_model.predict(x_test)
pred_y = clf.predict(test_x)
results = confusion_matrix(test_Y, pred_y)
error = zero_one_loss(test_Y, pred_y)
target_names = ['benign 0', 'dos 1', 'probe 2', 'r2l 3', 'u2r 4']

print("test result: \n" , results)
print("test error:", error)
# print("validation:", accuracy_score(test_Y, pred_y))
print("test validation:", accuracy_score(test_Y, pred_y))
print(classification_report(test_Y, pred_y, target_names=target_names))




train_x         0         1         2    3         4         5         6         7    \
0 -0.110249 -0.007679 -0.004919    0 -0.089486 -0.007736 -0.095076 -0.027023   
1 -0.110249 -0.007737 -0.004919    0 -0.089486 -0.007736 -0.095076 -0.027023   
2 -0.110249 -0.007762 -0.004919    0 -0.089486 -0.007736 -0.095076 -0.027023   
3 -0.110249 -0.007723 -0.002891    0 -0.089486 -0.007736 -0.095076 -0.027023   
4 -0.110249 -0.007728 -0.004814    0 -0.089486 -0.007736 -0.095076 -0.027023   

   8         9    ...  105  106  107  108  109  110  111  112  113  114  
0    0 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  
1    0 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  
2    0 -0.011664  ...    0    0    0    0    0    0    0    1    0    0  
3    1 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  
4    1 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  

[5 rows x 115 columns]
train_y    0
0  0
1  0
2  1
3  0
4  0
test_